## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-22-15-44-10 +0000,bbc,How Lib Dems hope 'punchy' new strategy will g...,https://www.bbc.com/news/videos/c4g7p03y715o?a...
1,2025-09-22-15-40-31 +0000,nyt,"Super Typhoon Ragasa Hits the Philippines, Hea...",https://www.nytimes.com/2025/09/22/world/asia/...
2,2025-09-22-15-38-15 +0000,bbc,Sarah Ferguson dropped from three charities ov...,https://www.bbc.com/news/articles/cddmr6v0jpzo...
3,2025-09-22-15-34-19 +0000,nyt,Hillary Clinton Sees a Dangerous Moment for Wo...,https://www.nytimes.com/2025/09/22/us/politics...
4,2025-09-22-15-33-38 +0000,bbc,Bodycare to shut remaining stores with loss of...,https://www.bbc.com/news/articles/clyd9rk3ljko...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2313/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
88,trump,42
54,kirk,34
83,charlie,27
84,memorial,16
433,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
81,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...,144
184,2025-09-21-22-51-23 +0000,nypost,Trump calls Charlie Kirk one of ‘America’s gre...,https://nypost.com/2025/09/21/us-news/trump-ca...,144
176,2025-09-21-23-30-00 +0000,wsj,President Trump and the upper echelon of his a...,https://www.wsj.com/us-news/charlie-kirk-funer...,139
145,2025-09-22-04-29-00 +0000,wsj,The modern conservative movement arrived in Ar...,https://www.wsj.com/politics/policy/charlie-ki...,138
9,2025-09-22-15-26-06 +0000,nypost,The 5 most emotional moments of the Charlie Ki...,https://nypost.com/2025/09/22/us-news/the-five...,137


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
81,144,2025-09-22-11-14-50 +0000,nypost,Charlie Kirk inspired Trump and Elon Musk to r...,https://nypost.com/2025/09/22/us-news/charlie-...
185,51,2025-09-21-22-27-41 +0000,nypost,Erika Kirk forgives assassin who killed husban...,https://nypost.com/2025/09/21/us-news/erika-ki...
134,35,2025-09-22-06-35-15 +0000,bbc,South Korea would accept a Trump-Kim deal to f...,https://www.bbc.com/news/articles/cwy91w0e1z2o...
37,32,2025-09-22-14-13-09 +0000,nypost,John Oliver blasts Disney boss Bob Iger over J...,https://nypost.com/2025/09/22/business/john-ol...
138,32,2025-09-22-06-10-20 +0000,nypost,Two 25-year-old elk hunters found dead with no...,https://nypost.com/2025/09/22/us-news/two-elk-...
102,31,2025-09-22-10-00-00 +0000,nypost,NJ governor’s race heats up as Ciattarelli’s G...,https://nypost.com/2025/09/22/us-news/the-kama...
94,29,2025-09-22-10-32-08 +0000,nypost,California high school student suspended after...,https://nypost.com/2025/09/22/us-news/californ...
210,29,2025-09-21-20-00-00 +0000,wsj,The Trump administration’s moves to upend the ...,https://www.wsj.com/politics/policy/trumps-100...
20,26,2025-09-22-14-53-29 +0000,nypost,Sarah Ferguson dumped by children’s hospice ch...,https://nypost.com/2025/09/22/world-news/sarah...
235,24,2025-09-21-15-56-12 +0000,bbc,Young Lib Dems on whether their party could le...,https://www.bbc.com/news/videos/cp3vv2g2yvdo?a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
